# More Functions
Not all functions are in the `nexuscli` python client (but they will be eventually!)

Even so, not all clients will be using python. All of the functions are available via plain HTTP and can be accessed from any client that can issue HTTP requests. Like `python`, `curl`, `IDL`, `MATLAB`, `Java`, `...`

## Listing the Capabilities

There is an endpoint exposed on the server called `/capabilities` which provides details about the current algorithms that are available from the server.

### TODOs

1. Target your EC2 instance



In [ ]:
import requests
import pprint

# TODO Target your EC2 instance
host = "http://<public dns>:8083"

request = "{}/capabilities".format(host)

pprint.pprint(requests.get(request).json())

# Time Average Map

Time Average Map is one of the algorithms that have not been integrated into the Python client yet. This cell shows an example of how you would build an HTTP request to run a Time Average Map computation.

This particular example is of a Time Average Map of the El Nino 3.4 region overlayed on an image of the Blue Marble. 

Target your EC2 instance before attempting to run this cell.

## TODOs

1. Target your EC2 instance

In [ ]:
import requests
import json
import datetime
import time
import numpy
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from shapely.geometry import box
from mpl_toolkits.basemap import Basemap

epoch = datetime.datetime.utcfromtimestamp(0)

# TODO Target your EC2 instance
host = "http://<public dns>:8083"

# Build HTTP request
ds='AVHRR_OI_L4_GHRSST_NCEI'
bbox = box(-170, -5, -120, 5) #minx, miny, maxx, maxy
date_format = '%Y-%m-%dT%H:%M:%SZ'
startTime = int((datetime.datetime(2016,1,1) - epoch).total_seconds())
endTime = int((datetime.datetime(2016,12,31) - epoch).total_seconds())
request = "{}/timeAvgMapSpark?ds={}&startTime={}&endTime={}" \
            "&minLon={}&minLat={}&maxLon={}&maxLat={}&spark=local,16,32" \
    .format(host, ds, startTime, endTime, *bbox.bounds)
print (request)

# Send request to server
response = requests.get(request).json()

# Parse the response
lons = [point['lon'] for point in response['data'][0]]
lats = [a_list[0]['lat'] for a_list in response['data']]

my_list = numpy.ndarray((len(lats), len(lons)))
for x in range(0, len(lats)):
    for y in range(0, len(lons)):
        my_list[x][y] = response['data'][x][y]['avg']

# Generate an image of the data using a rainbow colorbar
fig, ax1 = plt.subplots(figsize=(16,8), dpi=100)
im = ax1.pcolormesh(lons, lats, my_list, vmin=my_list.min(),vmax=my_list.max(), cmap='gist_rainbow')
fig.colorbar(im, ax=ax1)

# Display it on a map
map = Basemap()
map.bluemarble(scale=0.5)